In [2]:
!pip install groq tenacity sentence-transformers scikit-learn tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.0 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/sooo66/semeval2026-task12-dataset.git

Cloning into 'semeval2026-task12-dataset'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 69 (delta 30), reused 51 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 6.72 MiB | 21.36 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [10]:
import json
import os

# ==========================================
# تنظیمات مسیر فایل‌ها
# ==========================================
# مسیر فایل‌های از پیش پردازش شده (قدیمی و جدید)
OLD_PROCESSED_FILE = '/kaggle/input/datasets/daniyalghoreyshi/out111/train_data_enriched_with_cot.jsonl'
RECENT_PROCESSED_FILE = '/kaggle/input/datasets/daniyalghoreyshi/newout/train_data_enriched_with_cot (4).jsonl'

# مسیر فایل خروجی نهایی و تمیز برای فاین‌تیونینگ
OUTPUT_FILE = 'train_data_ready_for_finetuning.jsonl'

# ==========================================
# پردازش و ادغام
# ==========================================
valid_samples = {}

print("در حال خواندن و بررسی فایل‌های داده...")

# خواندن هر دو فایل
for filepath in [OLD_PROCESSED_FILE, RECENT_PROCESSED_FILE]:
    if os.path.exists(filepath):
        print(f"✅ در حال استخراج از فایل: {filepath}")
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                if not line.strip(): 
                    continue
                try:
                    data = json.loads(line)
                    # فیلتر کردن: فقط داده‌هایی که استدلال دارند و متن آن‌ها خطا نیست
                    if data.get('reasoning') and data.get('reasoning') != "خطا در تولید استدلال":
                        # استفاده از ID به عنوان کلید دیکشنری باعث می‌شود 
                        # اگر یک نمونه در هر دو فایل بود، به صورت خودکار تکرارگیری (Deduplication) شود.
                        valid_samples[data['id']] = data
                except json.JSONDecodeError:
                    continue
    else:
        print(f"⚠️ اخطار: فایل پیدا نشد (مسیر را بررسی کنید) -> {filepath}")

# ==========================================
# ذخیره فایل نهایی
# ==========================================
print(f"\nتعداد کل نمونه‌های سالم، خالص و بدون تکرار: {len(valid_samples)}")
print(f"در حال ذخیره در فایل نهایی ({OUTPUT_FILE})...")

with open(OUTPUT_FILE, 'w', encoding='utf-8') as out_file:
    for sample in valid_samples.values():
        out_file.write(json.dumps(sample, ensure_ascii=False) + '\n')

print("🎉 عملیات با موفقیت به پایان رسید!")
print("اکنون دیتاست شما کاملاً برای مرحله آموزش (QLoRA Fine-Tuning) آماده است.")

در حال خواندن و بررسی فایل‌های داده...
✅ در حال استخراج از فایل: /kaggle/input/datasets/daniyalghoreyshi/out111/train_data_enriched_with_cot.jsonl
✅ در حال استخراج از فایل: /kaggle/input/datasets/daniyalghoreyshi/newout/train_data_enriched_with_cot (4).jsonl

تعداد کل نمونه‌های سالم، خالص و بدون تکرار: 1461
در حال ذخیره در فایل نهایی (train_data_ready_for_finetuning.jsonl)...
🎉 عملیات با موفقیت به پایان رسید!
اکنون دیتاست شما کاملاً برای مرحله آموزش (QLoRA Fine-Tuning) آماده است.


In [3]:
# !pip install groq tenacity sentence-transformers scikit-learn tqdm

import json
import re
import os
import time
from tqdm import tqdm
from groq import Groq
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ==========================================
# تنظیمات اولیه و API
# ==========================================
# کلید API خود را اینجا قرار دهید
os.environ["GROQ_API_KEY"] = ""
client = Groq()

# استفاده از مدل سریع‌تر برای جلوگیری از خطای Rate Limit
GROQ_MODEL = "llama-3.1-8b-instant"

# مسیر فایل‌های آموزش (داده‌های خام)
TRAIN_QUESTIONS_FILE = 'semeval2026-task12-dataset/train_data/questions.jsonl'
TRAIN_DOCS_FILE = 'semeval2026-task12-dataset/train_data/docs.json'

# مسیر فایل‌های از پیش پردازش شده (قدیمی و جدید)
OLD_PROCESSED_FILE = '/kaggle/input/datasets/daniyalghoreyshi/out111/train_data_enriched_with_cot.jsonl'
RECENT_PROCESSED_FILE = '/kaggle/input/datasets/daniyalghoreyshi/newout/train_data_enriched_with_cot (4).jsonl' # مسیر فایلی که به تازگی آپلود کردید

# مسیر فایل خروجی نهایی و تمیز
OUTPUT_FILE = 'train_data_enriched_with_cot_final.jsonl'

# ==========================================
# توابع پیش‌پردازش و RAG
# ==========================================
print("در حال بارگذاری مدل Embedding...")
embedder = SentenceTransformer('BAAI/bge-large-en-v1.5')

def clean_text(text):
    if not text or not isinstance(text, str): return ""
    return re.sub(r'\s+', ' ', text).strip()

def load_and_merge_data(questions_path, docs_path):
    with open(docs_path, 'r', encoding='utf-8') as f:
        docs_raw = json.load(f)
    
    docs_dict = {}
    if isinstance(docs_raw, list):
        for item in docs_raw:
            if not isinstance(item, dict): continue
            t_id = item.get("topic_id")
            if not t_id: continue
            if t_id not in docs_dict: docs_dict[t_id] = []
            
            if "docs" in item or "documents" in item:
                inner_docs = item.get("docs", item.get("documents", []))
                for d in inner_docs:
                    if isinstance(d, dict): docs_dict[t_id].append(d.get("text", d.get("content", d.get("doc", ""))))
                    elif isinstance(d, str): docs_dict[t_id].append(d)
            else:
                text_val = item.get("text", item.get("content", item.get("doc", "")))
                if text_val: docs_dict[t_id].append(text_val)

    merged = []
    with open(questions_path, 'r', encoding='utf-8') as f:
        for line in f:
            if not line.strip(): continue
            q_item = json.loads(line.strip())
            t_id = q_item.get("topic_id")
            related_docs = docs_dict.get(t_id, [])
            cleaned_docs = [clean_text(doc) for doc in related_docs if doc]
            
            merged.append({
                "id": q_item.get("id"),
                "topic_id": t_id,
                "target_event": clean_text(q_item.get("target_event")),
                "options": {
                    "A": clean_text(q_item.get("option_A")),
                    "B": clean_text(q_item.get("option_B")),
                    "C": clean_text(q_item.get("option_C")),
                    "D": clean_text(q_item.get("option_D"))
                },
                "golden_answer": q_item.get("golden_answer"),
                "context": " ".join(cleaned_docs)
            })
    return merged

def retrieve_relevant_context(sample, top_k=6):
    context = sample['context']
    sentences = re.split(r'(?<=[.!?])\s+', context)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 20]
    if not sentences: return ""

    query = sample['target_event'] + " " + " ".join(sample['options'].values())
    sentence_embeddings = embedder.encode(sentences)
    query_embedding = embedder.encode([query])

    similarities = cosine_similarity(query_embedding, sentence_embeddings).flatten()
    top_indices = similarities.argsort()[-top_k:]
    return " ".join([sentences[i] for i in sorted(top_indices)])

# ==========================================
# تابع تماس با Groq API همراه با Retry خودکار 
# ==========================================
@retry(wait=wait_exponential(multiplier=1, min=4, max=60), stop=stop_after_attempt(5))
def generate_reasoning_with_groq(sample, filtered_context):
    
    golden_letters = sample['golden_answer'].split(',')
    golden_texts = [sample['options'][letter.strip()] for letter in golden_letters]
    golden_combined_text = " AND ".join(golden_texts)
    
    prompt = f"""You are an expert logical reasoning AI. 
Based ONLY on the provided Context, explain logically why the following event:
"{golden_combined_text}"
is the direct cause of the Target Event:
"{sample['target_event']}"

Context: {filtered_context}

CRITICAL RULES:
1. DO NOT use the words "Option", "Choice", "A", "B", "C", or "D".
2. Act as an independent analyst deducing the cause from the text.
3. Keep it brief (2 to 3 sentences).
4. Output ONLY the reasoning text."""

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=GROQ_MODEL,
        temperature=0.1,
        max_tokens=150,
    )
    
    return chat_completion.choices[0].message.content.strip()

# ==========================================
# حلقه اصلی پردازش و ذخیره‌سازی
# ==========================================
print("در حال بارگذاری داده‌های آموزشی خام...")
train_dataset = load_and_merge_data(TRAIN_QUESTIONS_FILE, TRAIN_DOCS_FILE)
print(f"تعداد کل نمونه‌های آموزشی: {len(train_dataset)}")

print("در حال تجمیع و بررسی داده‌های پردازش شده قبلی...")
valid_samples = {}

# خواندن فایل قدیمی و فایل جدید و فیلتر کردن خطادارها
for filepath in [OLD_PROCESSED_FILE, RECENT_PROCESSED_FILE]:
    if os.path.exists(filepath):
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                if not line.strip(): continue
                data = json.loads(line)
                # شرط مهم: فقط نمونه‌هایی که استدلال دارند و متن آن خطا نیست را بپذیر
                if data.get('reasoning') and data.get('reasoning') != "خطا در تولید استدلال":
                    valid_samples[data['id']] = data

print(f"تعداد {len(valid_samples)} نمونه با موفقیت و بدون خطا از قبل پردازش شده‌اند.")

# ایجاد فایل نهایی تمیز و ذخیره نمونه‌های سالم در آن
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for s in valid_samples.values():
        f.write(json.dumps(s, ensure_ascii=False) + '\n')

print("شروع پردازش نمونه‌های خطادار و نمونه‌های باقی‌مانده...")

# باز کردن فایل به صورت Append برای اضافه کردن نمونه‌های جدید
with open(OUTPUT_FILE, 'a', encoding='utf-8') as out_file:
    for sample in tqdm(train_dataset, desc="Generating Reasoning"):
        
        # اگر نمونه در دیکشنری نمونه‌های سالم وجود داشت، از آن عبور کن
        if sample['id'] in valid_samples:
            continue
            
        smart_context = retrieve_relevant_context(sample, top_k=6)
        
        try:
            reasoning = generate_reasoning_with_groq(sample, smart_context)
            time.sleep(1.5) 
        except Exception as e:
            print(f"خطا در پردازش ID {sample['id']}: {e}")
            reasoning = "خطا در تولید استدلال"
            time.sleep(5)
            
        enriched_sample = sample.copy()
        enriched_sample['reasoning'] = reasoning
        enriched_sample['context'] = smart_context 
        
        out_file.write(json.dumps(enriched_sample, ensure_ascii=False) + '\n')
        out_file.flush() 

print(f"\n✅ پردازش تمام شد. فایل بی‌نقص و نهایی در {OUTPUT_FILE} ذخیره شد.")

در حال بارگذاری مدل Embedding...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [9]:
import os
from groq import Groq

os.environ["GROQ_API_KEY"] = "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"
client = Groq()


try:
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": "Hello!"}],
        model="llama-3.3-70b-versatile",
        max_tokens=10
    )
    print("✅ ارتباط با Groq با موفقیت برقرار شد!")
except Exception as e:
    print(f"❌ جزئیات دقیق خطا: {e}")

❌ جزئیات دقیق خطا: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}
